# Import Libraries

In [1]:
import os
import zipfile
from zipfile import ZipFile 
import datetime 
import re
import json
from random import sample 

from pymongo import MongoClient 
import glob
import pandas as pd

# Create MongoDB Instance

In [2]:
# Making a Connection to MongoDB
#mongo = MongoClient("mongodb://localhost:27017/") # ?

Client = MongoClient() 
mongo = MongoClient('localhost', 27017) 

In [4]:
# Create a database called Task2 in MongoDB, we are going to store Task2 as an object in python called 'db'
db = mongo['Task2'] 
#mongo.drop_database('Task2')   # this is the code to drop the Database from Mongo!

In [21]:
# Get a list of all the databases in our MongoDB
for dbs in mongo.list_databases():
    print(dbs)

{'name': 'Task2', 'sizeOnDisk': 20275200.0, 'empty': False}
{'name': 'admin', 'sizeOnDisk': 40960.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 110592.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 40960.0, 'empty': False}


In [6]:
# create a collection in our database ("Task2" in Mongo) called 'data'.
# We store the 'data' collection as a python object named "Collection"
Collection = db["data"] 
#Collection.drop()  # this is the code to drop the Collection from the Database!

In [22]:
# List all the Collections within the python object 'db' (which contains our Mongo database called Task2)
# if you drop it, returns []. Just run Coll=db[data], still []. ADD to colleciton, now it shows ['data']
db.list_collection_names()

['data']

In [8]:
# show ONE document within the python object 'Collection' which is storing our Mongo Collection called 'data'
Collection.find_one()

## Take a peak into the Zip file to SEE the documents and folders inside

In [9]:
zipfile = "BIKES_DATASET.zip"
sons = []
# opening the zip file in READ mode 
with ZipFile(zipfile, 'r') as zip: 
    for info in zip.infolist(): 
        sons.append(info.filename)
        #print(info.filename) 
sons

['BIKES/',
 'BIKES/71/',
 'BIKES/71/71/',
 'BIKES/71/71/30/',
 'BIKES/71/71/30/7171306.json',
 'BIKES/71/50/',
 'BIKES/71/50/16/',
 'BIKES/71/50/16/7150169.json',
 'BIKES/71/46/',
 'BIKES/71/46/01/',
 'BIKES/71/46/01/7146014.json',
 'BIKES/71/54/',
 'BIKES/71/54/57/',
 'BIKES/71/54/57/7154576.json',
 'BIKES/71/89/',
 'BIKES/71/89/87/',
 'BIKES/71/89/87/7189874.json',
 'BIKES/71/57/',
 'BIKES/71/57/58/',
 'BIKES/71/57/58/7157581.json',
 'BIKES/71/43/',
 'BIKES/71/43/02/',
 'BIKES/71/43/02/714302.json',
 'BIKES/71/11/',
 'BIKES/71/11/95/',
 'BIKES/71/11/95/711195.json',
 'BIKES/71/69/',
 'BIKES/71/69/01/',
 'BIKES/71/69/01/7169014.json',
 'BIKES/71/07/',
 'BIKES/71/07/06/',
 'BIKES/71/07/06/7107063.json',
 'BIKES/71/74/',
 'BIKES/71/74/81/',
 'BIKES/71/74/81/7174810.json',
 'BIKES/71/51/',
 'BIKES/71/51/76/',
 'BIKES/71/51/76/7151760.json',
 'BIKES/71/39/',
 'BIKES/71/39/44/',
 'BIKES/71/39/44/7139449.json',
 'BIKES/71/91/',
 'BIKES/71/91/18/',
 'BIKES/71/91/18/7191188.json',
 'BIKES/71/

## Create a list called 'jsons' of all the json files in the entire folder

In [10]:
jsons = []
for j in sons:
    if j.endswith('.json') == True:
        jsons.append(j)
jsons

['BIKES/71/71/30/7171306.json',
 'BIKES/71/50/16/7150169.json',
 'BIKES/71/46/01/7146014.json',
 'BIKES/71/54/57/7154576.json',
 'BIKES/71/89/87/7189874.json',
 'BIKES/71/57/58/7157581.json',
 'BIKES/71/43/02/714302.json',
 'BIKES/71/11/95/711195.json',
 'BIKES/71/69/01/7169014.json',
 'BIKES/71/07/06/7107063.json',
 'BIKES/71/74/81/7174810.json',
 'BIKES/71/51/76/7151760.json',
 'BIKES/71/39/44/7139449.json',
 'BIKES/71/91/18/7191188.json',
 'BIKES/71/79/41/7179411.json',
 'BIKES/71/08/75/7108758.json',
 'BIKES/71/08/83/7108833.json',
 'BIKES/71/08/81/7108811.json',
 'BIKES/71/08/73/7108738.json',
 'BIKES/71/18/23/7118238.json',
 'BIKES/71/01/68/7101681.json',
 'BIKES/71/81/92/7181922.json',
 'BIKES/71/72/47/7172471.json',
 'BIKES/71/72/93/7172935.json',
 'BIKES/71/31/28/7131285.json',
 'BIKES/71/16/35/7116356.json',
 'BIKES/71/82/11/7182115.json',
 'BIKES/71/82/00/7182007.json',
 'BIKES/71/12/23/7112236.json',
 'BIKES/71/12/29/7112298.json',
 'BIKES/71/12/78/7112789.json',
 'BIKES/71

# For loop through the zip file and add each json to Mongo

In [11]:
with ZipFile(zipfile, 'r') as zip:
    for son in jsons:
        with zip.open(r''+son) as file: #, encoding="utf8"
            data = file.read()
            dice = json.loads(data.decode("utf-8"))
            # change price to numeric
            if dice['ad_data']['Price'] == 'Askforprice':
                dice['ad_data']['Price'] = float(0)
            else:
                dice['ad_data']['Price'] = float(dice['ad_data']['Price'][1:].replace('.','').replace(',',''))
            # change CC to numeric
            try:
                dice['ad_data']['Cubic capacity'] = float(dice['ad_data']['Cubic capacity'][:-3].replace('.','').replace(',',''))
            except:
                dice['ad_data']['Cubic capacity'] = float(0)
            Collection.insert_one(dice) 

In [12]:
# number of documents in the collection 
total_count = Collection.count_documents({}) 
print("Total number of documents : ", total_count) 

Total number of documents :  29701


## TASK 2.2
* How many bikes are there for sale?

We query the collection for type: Bikes. We see that ALL documents in the collection are of type Bike

In [14]:
query = {"metadata.type": "Bikes"}

# number of documents in the collection 
total_count = Collection.count_documents(query) 
print("Total number of Bikes : ", total_count) 

Total number of Bikes :  29701


## TASK 2.3
* What is the average price of a motorcycle (give a number)? What is the number of listings that were used in order to calculate this average (give a number as well)? Is the number of listings used the same as the answer in 2.2? Why?


Here we are classifying a "motorcycle" as any bike with > 50 CC.
The average is 3066.60 euros using 27,418 listings. This is not the same number as the total number of listings as it is a subset of the data

In [15]:
agg_result= Collection.aggregate([
                                # we filter CC to be > 50
                                { '$match': { 'ad_data.Cubic capacity': { '$gt' : 50 } } },
                                {"$group":
                                 #{"_id":"$ad_data.Category", "avg_price": # we could instead group by Category,
                                                                           # but none are "motorcycle"
                                 #{"_id":"$ad_data.Cubic capacity", "avg_price": # OR group by CC
                                  {"_id":"null", "avg_price": 
                                  {"$avg":"$ad_data.Price"}, "count": {"$sum":1}}}
                                  ] )

#Cubic capacity
for i in agg_result: 
    print(i)

{'_id': 'null', 'avg_price': 3066.603472171566, 'count': 27418}


## TASK 2.4
* What is the maximum and minimum price of a motorcycle currently available in the market?

In [16]:
agg_result= Collection.aggregate([
                                # > 50 CC
                                { '$match': { 'ad_data.Cubic capacity': { '$gt' : 50 } } },
                                # Price > 25 euros i.e. "realistic" listings
                                { '$match': { 'ad_data.Price': { '$gt' : 25 } } },
                                {"$group":
                                  {"_id":"null", 
                                   "min_price": {"$min":"$ad_data.Price"}, 
                                   "max_price": {"$max":"$ad_data.Price"}}}
                                  ] )

#Cubic capacity
for i in agg_result: 
    print(i)

{'_id': 'null', 'min_price': 30.0, 'max_price': 89000.0}


## TASK 2.5
* How many listings have a price that is identified as negotiable?

None, there were a few listings that had price "Askforprice", but none that were listed as "Negotiable"

In [17]:
query = {"ad_data.Price": "Negotiable"}

# number of documents in the collection 
total_count = Collection.count_documents(query) 
print("Total number of documents : ", total_count) 

Total number of documents :  0


In [19]:
# here we ask mongo of a list of all the unique values for this field
Collection.distinct( "ad_data.Price" )

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 15.0,
 16.0,
 17.0,
 20.0,
 22.0,
 25.0,
 30.0,
 35.0,
 40.0,
 50.0,
 51.0,
 60.0,
 70.0,
 80.0,
 99.0,
 100.0,
 101.0,
 111.0,
 118.0,
 120.0,
 125.0,
 130.0,
 140.0,
 145.0,
 149.0,
 150.0,
 170.0,
 175.0,
 180.0,
 190.0,
 199.0,
 200.0,
 210.0,
 220.0,
 230.0,
 240.0,
 245.0,
 249.0,
 250.0,
 260.0,
 270.0,
 280.0,
 290.0,
 298.0,
 299.0,
 300.0,
 310.0,
 311.0,
 320.0,
 330.0,
 333.0,
 340.0,
 350.0,
 360.0,
 370.0,
 380.0,
 390.0,
 399.0,
 400.0,
 410.0,
 420.0,
 425.0,
 429.0,
 430.0,
 440.0,
 450.0,
 460.0,
 465.0,
 470.0,
 480.0,
 490.0,
 495.0,
 499.0,
 500.0,
 520.0,
 530.0,
 545.0,
 549.0,
 550.0,
 555.0,
 560.0,
 567.0,
 570.0,
 580.0,
 590.0,
 599.0,
 600.0,
 620.0,
 630.0,
 640.0,
 645.0,
 649.0,
 650.0,
 660.0,
 670.0,
 680.0,
 690.0,
 699.0,
 700.0,
 701.0,
 720.0,
 730.0,
 740.0,
 745.0,
 748.0,
 749.0,
 750.0,
 751.0,
 760.0,
 775.0,
 780.0,
 790.0,
 795.0,
 799.0,
 800.0,
 820.0,
 830.0,
 840

In [20]:
prices = pd.DataFrame(Collection.distinct( "ad_data.Price" ))
prices.columns = ['price']
# here we ask pandas to convert to float64; if this succeeds, then we know all values are numeric
prices['price'] = prices['price'].astype('float64')
prices

,price
0,0.0
1,1.0
2,2.0
3,3.0
4,4.0
...,...
955,43000.0
956,46500.0
957,58000.0
958,60000.0
